In [508]:
import pandas as pd
import seaborn as sns 
# sns.get_dataset_names()

In [509]:
#! pip install mlxtend

In [510]:
tips=sns.load_dataset('tips')
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [511]:
tips=pd.get_dummies(tips)
tips

,total_bill,tip,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,1.01,2,0,1,0,1,0,0,0,1,0,1
1,10.34,1.66,3,1,0,0,1,0,0,0,1,0,1
2,21.01,3.50,3,1,0,0,1,0,0,0,1,0,1
3,23.68,3.31,2,1,0,0,1,0,0,0,1,0,1
4,24.59,3.61,4,0,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,3,1,0,0,1,0,0,1,0,0,1
240,27.18,2.00,2,0,1,1,0,0,0,1,0,0,1
241,22.67,2.00,2,1,0,1,0,0,0,1,0,0,1
242,17.82,1.75,2,1,0,0,1,0,0,1,0,0,1


In [512]:
tips1= tips.drop(columns=['sex_Female','smoker_No','day_Thur'])
tips1.columns

Index(['total_bill', 'tip', 'size', 'sex_Male', 'smoker_Yes', 'day_Fri',
       'day_Sat', 'day_Sun', 'time_Lunch', 'time_Dinner'],
      dtype='object')

In [513]:
X= tips1.drop(columns="tip", axis=1)
X.shape

(244, 9)

In [514]:
y= tips1["tip"]
y.shape

(244,)

In [515]:
from sklearn.model_selection import train_test_split

In [516]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=19)

In [517]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [518]:
from sklearn.linear_model import ElasticNet

elast_net = ElasticNet()

In [519]:
elast_net.fit(X_train, y_train)



ElasticNet()

In [520]:
from sklearn.metrics import mean_absolute_error , mean_squared_error, r2_score

In [521]:
y_pred = elast_net.predict(X_test)

In [522]:
mean_absolute_error(y_test, y_pred)

1.214329254741493

In [523]:
mean_squared_error(y_test, y_pred)

2.899097972303772

In [524]:
r2_score(y_test, y_pred)

0.14011647876429634

In [525]:
param_grid={"alpha": [0.01, 0.02,0.3, 0.4, 0.7, 1], 'l1_ratio': [0.01, 0.02,0.3, 0.4, 0.7, 1]}

In [526]:
from sklearn.model_selection import GridSearchCV

In [527]:
elastic_cv = GridSearchCV(elast_net, param_grid, cv=5 , scoring ="neg_mean_squared_error", n_jobs=2)

In [528]:
elastic_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=2,
             param_grid={'alpha': [0.01, 0.02, 0.3, 0.4, 0.7, 1],
                         'l1_ratio': [0.01, 0.02, 0.3, 0.4, 0.7, 1]},
             scoring='neg_mean_squared_error')

In [529]:
y_pred = elastic_cv.predict(X_test)

In [530]:
mean_absolute_error(y_test, y_pred)

0.9264507139769552

In [531]:
mean_squared_error(y_test, y_pred)

1.831790178307955

In [532]:
r2_score(y_test, y_pred)

0.45668404319680656

In [533]:
elastic_cv.best_estimator_

ElasticNet(alpha=0.02, l1_ratio=1)

In [534]:
elastic_cv.best_estimator_.intercept_.round(2)

2.96

In [535]:
elastic_cv.best_estimator_.coef_

array([ 0.67031623,  0.16803338, -0.04762338, -0.0319659 ,  0.02493616,
       -0.02515258,  0.02778364, -0.        ,  0.        ])

In [536]:
feature_names=['total_bill',  'size', 'sex_Male', 'smoker_Yes', 'day_Fri',
       'day_Sat', 'day_Sun', 'time_Lunch', 'time_Dinner']

In [537]:
import pandas as pd
df1 = pd.DataFrame({'Zmienna':feature_names, 'Coef': elastic_cv.best_estimator_.coef_.round(2)})
df1

,Zmienna,Coef
0,total_bill,0.67
1,size,0.17
2,sex_Male,-0.05
3,smoker_Yes,-0.03
4,day_Fri,0.02
5,day_Sat,-0.03
6,day_Sun,0.03
7,time_Lunch,-0.00
8,time_Dinner,0.00


In [538]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [539]:
X= tips1.drop(columns="tip", axis=1)
y= tips1["tip"]

In [540]:
from sklearn.model_selection import train_test_split

In [541]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=19)

In [542]:
lr.fit(X_train, y_train)

LinearRegression()

In [543]:
from sklearn.metrics import mean_absolute_error , mean_squared_error, r2_score

In [544]:
y_pred = lr.predict(X_test)

In [545]:
mean_absolute_error(y_test, y_pred)

0.8683485514240518

In [546]:
mean_squared_error(y_test, y_pred)

1.6130323050447053

In [547]:
r2_score(y_test, y_pred)

0.5215684631635309

In [548]:
lr.intercept_.round(2)

0.88

In [549]:
lr.coef_

array([ 0.0824646 ,  0.1941934 , -0.15087754, -0.1185585 ,  0.29951908,
        0.07512996,  0.22951647,  0.06271763, -0.06271763])

In [550]:
import pandas as pd
df1 = pd.DataFrame({'Zmienna':feature_names, 'Coef': lr.coef_.round(2)})
df1

,Zmienna,Coef
0,total_bill,0.08
1,size,0.19
2,sex_Male,-0.15
3,smoker_Yes,-0.12
4,day_Fri,0.30
5,day_Sat,0.08
6,day_Sun,0.23
7,time_Lunch,0.06
8,time_Dinner,-0.06


In [551]:
# regresja krokowa

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs = SFS(lr, 
          k_features="best", 
          forward=True, 
          floating=False, 
          scoring='neg_mean_squared_error',
          cv=5)

sfs.fit(X, y)

# Get selected feature names
selected_features = list(sfs.k_feature_names_)
print("Selected features:", selected_features)

Selected features: ['total_bill', 'size']
